In [1]:
from bs4 import BeautifulSoup
import requests
import sys
import traceback

def get_text (url):
    buf = []
    
    try:
        soup = BeautifulSoup(requests.get(url).text, "html.parser")
        
        for p in soup.find_all("p"):
            buf.append(p.get_text())

        return "\n".join(buf)
    except:
        print(traceback.format_exc())
        sys.exit(-1)

In [6]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [17]:
url = "https://derwen.ai/policy"
#url = "https://www.congress.gov/bill/115th-congress/house-bill/4174/text"
text = get_text(url)

In [18]:
doc = nlp(text)

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)


 
 SPACE _SP  
 False False
Derwen Derwen PROPN NNP nsubj Xxxxx True False
, , PUNCT , punct , False False
Inc. Inc. PROPN NNP appos Xxx. False False
, , PUNCT , punct , False False
develops develop VERB VBZ ROOT xxxx True False
its -PRON- DET PRP$ poss xxx True True
products product NOUN NNS dobj xxxx True False
and and CCONJ CC cc xxx True True
services service NOUN NNS conj xxxx True False
based base VERB VBN prep xxxx True False
on on ADP IN prep xx True True
a a DET DT det x True True

 
 SPACE _SP  
 False False
privacy privacy NOUN NN pobj xxxx True False
by by ADP IN prep xx True True
design design NOUN NN nmod xxxx True False

 
 SPACE _SP  
 False False
approach approach NOUN NN pobj xxxx True False
. . PUNCT . punct . False False

 
 SPACE _SP  
 False False
Our -PRON- DET PRP$ poss Xxx True True
efforts effort NOUN NNS nsubjpass xxxx True False
are be VERB VBP auxpass xxx True True
intended intend VERB VBN ROOT xxxx True False
to to PART TO aux xx True True
be be VERB VB x

Occasionally occasionally ADV RB advmod Xxxxx True False
, , PUNCT , punct , False False
at at ADP IN prep xx True True
our -PRON- DET PRP$ poss xxx True True
discretion discretion NOUN NN pobj xxxx True False
, , PUNCT , punct , False False
we -PRON- PRON PRP nsubj xx True True
may may VERB MD aux xxx True True
include include VERB VB ROOT xxxx True False
or or CCONJ CC cc xx True True
offer offer VERB VB conj xxxx True False
third third ADJ JJ amod xxxx True True
- - PUNCT HYPH punct - False False
party party NOUN NN nmod xxxx True False

 
 SPACE _SP  
 False False
products product NOUN NNS dobj xxxx True False
or or CCONJ CC cc xx True True
services service NOUN NNS conj xxxx True False
on on ADP IN prep xx True True
our -PRON- DET PRP$ poss xxx True True
website website NOUN NN pobj xxxx True False
. . PUNCT . punct . False False

 
 SPACE _SP  
 False False
These these DET DT det Xxxxx True True
third third ADJ JJ amod xxxx True True
- - PUNCT HYPH punct - False False
party party

) ) PUNCT -RRB- punct ) False False
browser browser NOUN NN compound xxxx True False
mechanism mechanism NOUN NN nsubj xxxx True False
is be VERB VBZ advcl xx True True
in in ADP IN prep xx True True

 
 SPACE _SP  
 False False
place place NOUN NN pobj xxxx True False
. . PUNCT . punct . False False

 
 SPACE _SP  
 False False
DNT dnt NOUN NN nsubj XXX True False
represents represent VERB VBZ ROOT xxxx True False
a a DET DT det x True True
complete complete ADJ JJ amod xxxx True False
and and CCONJ CC cc xxx True True
explicit explicit ADJ JJ conj xxxx True False
opt opt NOUN NN dobj xxx True False
out out PART RP prt xxx True True
for for ADP IN prep xxx True True
any any DET DT det xxx True True
data datum NOUN NNS nmod xxxx True False

 
 SPACE _SP  
 False False
collection collection NOUN NN pobj xxxx True False
or or CCONJ CC cc xx True True
other other ADJ JJ amod xxxx True True
stateful stateful ADJ JJ amod xxxx True False
interactions interaction NOUN NNS conj xxxx True False

an an DET DT det xx True True
anonymous anonymous ADJ JJ amod xxxx True False
viewer viewer NOUN NN pobj xxxx True False
of of ADP IN prep xx True True
this this DET DT det xxxx True True
site site NOUN NN pobj xxxx True False

 
 SPACE _SP  
 False False
-- -- PUNCT : punct -- False False
you -PRON- PRON PRP nsubj xxx True True
agree agree VERB VBP ROOT xxxx True False
to to PART TO aux xx True True
observe observe VERB VB xcomp xxxx True False
the the DET DT det xxx True True
rights right NOUN NNS dobj xxxx True False
asserted assert VERB VBN acl xxxx True False
for for ADP IN prep xxx True True
these these DET DT det xxxx True True
licensed license VERB VBN amod xxxx True False
works work NOUN NNS pobj xxxx True False
. . PUNCT . punct . False False

 
 SPACE _SP  
 False False
Moreover moreover ADV RB advmod Xxxxx True True
, , PUNCT , punct , False False
you -PRON- PRON PRP nsubj xxx True True
agree agree VERB VBP ROOT xxxx True False
that that ADP IN mark xxxx True True
any any D

expelled expel VERB VBN conj xxxx True False
from from ADP IN prep xxxx True True
an an DET DT det xx True True
event event NOUN NN pobj xxxx True False
without without ADP IN prep xxxx True True
a a DET DT det x True True
refund refund NOUN NN pobj xxxx True False

 
 SPACE _SP  
 False False
and and CCONJ CC cc xxx True True
banned ban VERB VBN conj xxxx True False
from from ADP IN prep xxxx True True
future future ADJ JJ amod xxxx True False
engagement engagement NOUN NN pobj xxxx True False
at at ADP IN prep xx True True
the the DET DT det xxx True True
discretion discretion NOUN NN pobj xxxx True False
of of ADP IN prep xx True True
Derwen Derwen PROPN NNP pobj Xxxxx True False
, , PUNCT , punct , False False
Inc. Inc. PROPN NNP appos Xxx. False False



 


 SPACE _SP  


 False False
Please please INTJ UH intj Xxxxx True True
bring bring VERB VB ROOT xxxx True False
any any DET DT det xxx True True
concerns concern NOUN NNS dobj xxxx True False
to to ADP IN dative xx True True
t

In [19]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Derwen, Inc. 1 13 ORG
one 233 236 CARDINAL
PII 746 749 ORG
PII 773 776 ORG
United States 794 807 GPE
Derwen, Inc. 1583 1595 ORG
Help Menu 2528 2537 PERSON
Occasionally 2885 2897 PERSON
third 2942 2947 ORDINAL
third 2997 3002 ORDINAL
third 3261 3266 ORDINAL
third 3331 3336 ORDINAL
first 3381 3386 ORDINAL
the United States 3400 3417 GPE
California 3533 3543 GPE
US 3584 3586 GPE
PII 3649 3652 ORG
California 3658 3668 GPE
CalOPPA 3980 3987 ORG
the age of 13 years old 4094 4117 DATE
COPPA 4165 4170 ORG
The Federal Trade Commission 4197 4225 ORG
US 4271 4273 GPE
the COPPA Rule 4286 4300 LAW
under the age of 13 years old 4467 4496 DATE
the United States 4574 4591 GPE
Fair Information Practices 4912 4938 ORG
the Individual Redress Principle 5032 5064 ORG
CAN-SPAM Act 5470 5482 LAW
CAN-SPAM 5778 5786 ORG
DNT 6190 6193 ORG
DNS 6670 6673 ORG
HTTPS 6681 6686 PERSON
HTTP Strict Transport Security 7004 7034 ORG
Adam Scott 7311 7321 PERSON
Derwen, Inc. 7437 7449 ORG
Derwen, Inc. 8237 8249 ORG
Derwen,

In [20]:
for chunk in doc.noun_chunks:
    print(chunk.text)


Derwen
Inc.
its products
services
a
privacy
design
approach
Our efforts
privacy
the default
We
these practices
our designs
the one hand
visibility
transparency
the other hand
full lifecycle protection
end
our policies
respect
customer privacy
fact
our products
services
response
the
general ineffectiveness
online marketing
undue
emphasis
consumer surveillance
This privacy policy
who
their
Personally Identifiable Information
PII
PII
United States privacy law
information
security
information
other
information
contact
a single person
an individual
context
our privacy policy
a clear understanding
we
your PII
accordance
our website
our site
you
your name
email address
other details
you
your experience
addition
any agreements
account registration
API key
generation
legal documents
your IP address
browser agent info
timestamp
the
agreement
That data
Derwen
Inc.
who
an account
you
we
online materials
presentations
We
information
you
you
our site
an order
a notification
a form
information
our s

In [21]:
from spacy import displacy

displacy.render(doc, style="ent")

In [22]:
doc2 = nlp(u"The rain in Spain falls mainly on the plain.")
displacy.render(doc2, style="dep")